# __MODIS vs THAAO comparison of Integrated Water Vapor (IWV)__
[MODIS](https://modis.gsfc.nasa.gov/about/specifications.php): Moderate Resolution Imaging Spectroradiometer  <br>
[THAAO](https://www.thuleatmos-it.it/index.php): Thule High Arctic Atmospheric Observatory, Greenland  (coordinates  __76.5145, -68.7432__, altitude a.s.l. __220 m__)  <br>

__MXD05_L2 IWV product__ <br>
_References_ <br>
[MYD05_L2](https://atmosphere-imager.gsfc.nasa.gov/products/water-vapor) <br>

_Algorithm Theoretical Basis Documents (ATBDs)_ <br>
[ATBD](https://atmosphere-imager.gsfc.nasa.gov/documentation/atbds-plans-guides) <br>
[The MODIS Near-IR Water Vapor Algorithm](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/atbd_mod03_0.pdf) <br>
[MODIS Atmospheric Profile Retrieval
ATBD MOD07 Collection 6](https://modis-images.gsfc.nasa.gov/_docs/MOD07_atbd_v7_April2011.pdf) <br>
[Atmospheric Correction Algorithm: Spectral
Reflectances MOD08](https://modis.gsfc.nasa.gov/data/atbd/atbd_mod08.pdf) <br>
[MODIS Atmosphere L3 Gridded Product
(ATBD) & Users Guide](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/L3_ATBD_C6_C61_2019_02_20.pdf) <br>

__THAAO IWV product__ <br>
_References_ <br>
[The water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz](https://www.thuleatmos-it.it/data/vespa22/index.php)<br> 
[Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php)<br>
 

In [ ]:
import hvplot.pandas
import holoviews as hv
import numpy as np         
import pandas as pd
import panel as pn
import datetime
from holoviews.plotting.links import RangeToolLink
from sklearn.linear_model import LinearRegression

pd.options.plotting.backend = 'holoviews'
hv.extension('bokeh')

#### __MXD05_L2 and THAAO IWV products__ 

MODIS data were extracted as timeseries of IWV Near-IR data on a Point geometry centered on the THAAO Observatory from the MXD05_L2 products uploaded in the ADAM repository through the ADAM GUI. Within the framework of the RELIANCE project the repository has been populated of MODIS MXD05_L2 data covering the Greenland region upon our request. Other MODIS products have been requested and will be uploaded on the ADAM repository to explore other satellite products, including MODIS LST and AOD, and possibly validate them by close comparison with similar ground-based measurements routinely carried out at THAAO Observatory. The choice of the ADAM framework is motivated by possibility to upload several years of selected satelllite products on the repository, to easily extract temporal and regional subsets in the form of timeseries both through the ADAM GUI and the ADAM API developed within the RELIANCE project. The latter possibility in particular allows more flexibility to fully exploit the repository content and process the data by means of Jupyter notebooks and Python language, while the GUI is designed for an easier usage and allows straightforwrd dataset exploration and simpler data subset extraction. <br>

Thule (Greenland) IWV data timeseries have been downloaded from the [Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php) THAAO data access facility were the data are freely accessible. Likewise MODIS satellite IWV product, these data contain vertical column integrated amount  water vapor or precipitable water retrieved by means of the water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz (VESPA-22).  <br> 


#### __Loading MODIS and Thule timeseries__  
IWV timeseries of MODIS in .csv and THAAO in .txt formats are read into pandas dataframes. <br>
MODIS data have an additional 13% error column derived from the maximum error claimed in the NearIR IWV product ATBD (see sect. 3.1.4, page 16). <br>
THAAO IWV data are given in $kg/m^2$ units and thus divided for factor of 10 to have homogeoeous units with MODIS IWV data that are given in $cm$. <br>  

In [ ]:
def vespa_read(fname):
    thaao = pd.read_csv(fname, sep="\t", header=5, 
                      names=["time","IWV_THAAO"],
                      parse_dates=['time'],
                      date_parser=lambda col: pd.to_datetime(col))
    thaao['IWV_THAAO']=thaao['IWV_THAAO']/10.
    thaao['err_IWV']=thaao['IWV_THAAO']*0.05
    thaao=thaao.set_index('time', drop=False)
    thaao.index=thaao.index.rename('Time')
    thaao['time2']=thaao['time']
    thaao['key']=thaao.index
    return thaao

In [ ]:
def modis_read(fname):
    path_laads = fname
    modis = pd.read_csv(path_laads,header=0,sep='\t',
                      parse_dates=['time'],
                      date_parser=lambda col: pd.to_datetime(col, utc=True))
    modis['time'] = modis['time'].dt.tz_localize(None)
    modis['Err_IWV_MODIS'] = modis["IWV_MODIS"]*0.13
    modis = modis.set_index('time',drop=False)
    modis.index=modis.index.rename('Time')
    modis.head
    return modis

In [ ]:
def set_IO_dirs(path_segments):
    """ Final path_segments as a list, to be added to
    the baseline directory fo a complete absolute path
    to DATA and ELAB dirs.
    """
    from pathlib import Path
    
    # writing the path segments
    homedir = Path.home()          # similar to ~/
    data_dir_segments = [homedir,'WORK','DATA','Groundbased','THULE']
    [data_dir_segments.append(s) for s in path_segments]
    #elab_dir_segments = data_dir_segments
    #elab_dir_segments = data_dir_segments[:]
    import copy
    elab_dir_segments = copy.deepcopy(data_dir_segments)
    
    elab_dir_segments[2] = 'ELAB'
    return data_dir_segments, elab_dir_segments

def check_dirs_existence_mkdir(p_data, p_elab):
    """ Check out DATA and ELAB I/O dirs existence and 
    eventually creates them.
    """
    p_data.mkdir(parents=True, exist_ok=True)
    p_elab.mkdir(parents=True, exist_ok=True)
    return

def field_campaign_name():
    pass
    return fc_name

def sensor_name():
    pass
    return sensor_name



from pathlib import *

#transforming path segments into concrete paths
path_segments  = ['HATPRO']
path_data, path_elab = set_IO_dirs(path_segments)

print(f'data: {path_data}    id: {id(path_data)}')
print(f'elab: {path_elab}    id: {id(path_elab)}')

#p_data, p_elab = Path(*final_path_segments)
p_data = Path(*path_data)
p_elab = Path(*path_elab)

print(f'data: {p_data}    id: {id(p_data)}')
print(f'elab: {p_elab}    id: {id(p_elab)}')

check_dirs_existence_mkdir(p_data, p_elab)

In [ ]:
def hatpro_read(fname, year='2016'):
    #year = read_hatpro_year(fname)
    hatpro = pd.read_csv(fname, delim_whitespace=True, header=0)
    hatpro['Time'] = hatpro.JD_2016.to_frame().applymap(lambda x: 
                                                        datetime.datetime(
                                                            2016,1,1,0,0,0)+
                                                        datetime.timedelta(days=x-1.))
    hatpro.Time = hatpro.Time.dt.tz_localize(None)
    hatpro.set_index('Time', drop=True,inplace=True)
    hatpro.drop(columns=['JD_rif', 'JD_2016', 'mcorIWV', 'stdcoIWV', 'biasIWV',
                         'n_med','RF_med'], inplace=True)
    return hatpro


HATPRO = hatpro_read('/home/luca/WORK/DATA/Groundbased/THULE/HATPRO/IWV_2016_HATPRO.txt')

HATPRO

In [ ]:
HATPRO = hatpro_read('/home/luca/WORK/DATA/Groundbased/THULE/HATPRO/IWV_2017_HATPRO.txt')

In [ ]:
HATPRO

In [ ]:
VESPA = vespa_read('/home/luca/WORK/DATA/Groundbased/THULE/VESPA22/Vapor_20160716_20220311.txt')

In [ ]:
MOD_IR = modis_read('/home/luca/WORK/DATA/MODIS/THULE/Thule_LAADS_IWV_IR/MOD05_5km_totale.csv')

In [ ]:
MYD_IR = modis_read('/home/luca/WORK/DATA/MODIS/THULE/Thule_LAADS_IWV_IR/MYD05_5km_totale.csv')

In [ ]:
VESPA

### Drop rows with values >5 

In [ ]:
def df_drop(df, condition):
    df = df.drop(df[condition].index)
    return df

In [ ]:
MOD_IR = df_drop(MOD_IR, MOD_IR.IWV_MODIS > 5. )
MOD_IR = df_drop(MOD_IR, MOD_IR.longitude < -65000. )

MYD_IR = df_drop(MYD_IR, MYD_IR.IWV_MODIS > 5. )
MYD_IR = df_drop(MYD_IR, MYD_IR.longitude < -65000. )
number_MXD_data_points = len(MOD_IR.IWV_MODIS) + len(MYD_IR.IWV_MODIS)
MYD_IR.head(20)

In [ ]:
number_MXD_data_points = len(MOD_IR.IWV_MODIS) + len(MYD_IR.IWV_MODIS)

#### **MODIS and THAAO Integrated Water Vapor (IWV) timeseries**
##### Simple linear plots of the whole MODIS and THAAO IVW timeseries.

In [ ]:
MOD_IR['Err_IWV_MODIS']

In [ ]:
width = 1100
height = 300 
size = 3
scale = 0.3
color_MOD = 'cyan'
color_MYD = '#F2583E'
color_THAAO = 'green'

(MOD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                 height=height, width=width, size=size, 
                                 color=color_MOD, title ='IWV MOD')
* MOD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
+ MYD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD')
* MYD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
+ (MOD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                   height=height, width=width, size=size, 
                                   color=color_MOD,title ='IWV MOD')
* MOD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* MYD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD')
* MYD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
  ).opts(title='MOD+MYD')).cols(1)

In [ ]:
#MOD_IR #['Err_IWV_MODIS']

In [ ]:
thaao_timeseries = VESPA.hvplot.scatter(yaxis=True, 
                                        height=height, width=width, 
                                        size=size, 
                                        color=color_THAAO, 
                                        fields={'date': 'Date'},
                                        title='IWV THAAO - VESPA', label='VESPA',
                                        legend='top_right')
thaao_timeseries

Overlay of the IWV plots on the same time reference <br>

In [ ]:
MOD_timeseries = MOD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', 
                                                 ylabel='IWV (cm)',label='MOD',
                                                 height=height, width=width, 
                                                 size=size, color=color_MOD, 
                                                 title='IWV MOD - VESPA',
                                                 legend='top_right')
MYD_timeseries = MYD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV (cm)',label='MYD',
                                                 height=height, width=width, 
                                                 size=size, color=color_MYD, 
                                                 title='IWV MYD - VESPA',
                                                 legend='top_right')


(MOD_timeseries *thaao_timeseries
+ MYD_timeseries * thaao_timeseries
+ MOD_timeseries*MYD_timeseries * thaao_timeseries
* MOD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* MYD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)).cols(1)

In [ ]:
title = f'IWV MXD-VESPA                                         # data points={number_MXD_data_points}'
(MOD_timeseries*MYD_timeseries * thaao_timeseries).opts(title=title, show_grid=True)

### __TODO filtrare i dati MODIS con il QA__ !!!

#### __Merging pandas dataframes to filter MODIS and THAAO data points closest in time__

Due to time lenght of the timeseries, even if the notebook representation allows to zoom in into the datasets, it is still not easy to see how close in time two data points are. Since it makes little sense to compare two MODIS and THAAO IWV data points if they are too far apart, and they can easily be hours or days far apart, therefore easily mistaken if one relies solely on the visual inspection of the plots, a tool to select and compare only the data points which are closer in time is required. The following code allows to filter the data on a timespan basis by suitably merging modis and thaao pandas dataframes while defining the Timedelta parameter (expressed in s,m,d, i.e. seconds, minutes or days respectively) that allows to operate the time-based filter. Different choices of Timedelta allow to widen or narrow the time span around each MODIS data point and select for the comparison only the THAAO data points that fall within the chosen interval. This selection is based on MODIS data points because THAAO measures are far more dense. <br>  

In [ ]:
def closest_in_time(df1,df2,delta_time='30m'):
    """delta_time : string
    add RSME to new_df
    """
    newdf=pd.merge_asof(df1,df2,on='time',tolerance=pd.Timedelta(delta_time),direction='nearest')
    newdf['diff']=(newdf.time-newdf.time2).abs()
    newdf['diff_int']=newdf['diff'].astype(int)/1000000000
    newdf['diff_IWV']=(newdf.IWV_MODIS-newdf.IWV_THAAO)
    newdf['diff_perc_IWV']=(newdf.IWV_MODIS/newdf.IWV_THAAO)*100. - 100.
    newdf['diff_distance']=np.sqrt((newdf.latitude-76.5145)**2 + (newdf.longitude-68.7432)**2)
    newdf=newdf.sort_values('diff').drop_duplicates('key')
    newdf=newdf.set_index(['time','time2'], drop=False).sort_index()
    newdf.index=newdf.index.rename(['Time','Time2'])
    newdf=newdf[1:]
    newdf['RMSE_IWV']=((newdf.IWV_MODIS-newdf.IWV_THAAO)**2).mean()**.5
    newdf.info()
    newdf.head()
    return newdf

In [ ]:
deltatime = '10m'
newdf_MOD = closest_in_time(MOD_IR,VESPA, delta_time=deltatime)
newdf_MYD = closest_in_time(MYD_IR,VESPA, delta_time=deltatime)
number_MXD_data_points_deltatime = len(newdf_MOD.IWV_MODIS) + len(newdf_MYD.IWV_MODIS)

In [ ]:
newdf_MOD.head()
newdf_MOD.vza.max()

In [ ]:
newdf_TOT = pd.concat([newdf_MOD,newdf_MYD], sort=True)
#newdf_TOT.tail()

### Numero punti per mese 

In [ ]:
newdf_MOD.time.groupby([newdf_MOD['time'].dt.month.rename('month')]).agg({'count'}).transpose()

In [ ]:
newdf_MYD.time.groupby([newdf_MYD['time'].dt.month.rename('month')]).agg({'count'}).transpose()

### Timeseries su diff temporali 

In [ ]:

newdf_MOD_timeseries = newdf_MOD.IWV_MODIS.hvplot.scatter(x='Time', 
                                                          ylabel='IWV (cm)',
                                                          label='MOD',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_MOD, 
                                                          title= 'IWV MOD-VESPA',
                                                          legend='top_right')

newdf_MYD_timeseries = newdf_MYD.IWV_MODIS.hvplot.scatter(x='Time', ylabel='IWV (cm)',
                                                          label='IWV_MYD',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_MYD, 
                                                          title= 'IWV MYD-VESPA',
                                                          legend='top_right')

(newdf_MOD_timeseries*newdf_MOD.IWV_THAAO.hvplot.scatter(x='Time2',
                                                         height=height, 
                                                         width=width, 
                                                         size=size, 
                                                         color=color_THAAO)
+ newdf_MYD_timeseries*newdf_MYD.IWV_THAAO.hvplot.scatter(x='Time2',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_THAAO)
+ (newdf_MOD_timeseries *newdf_MYD_timeseries
* MOD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* MYD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)   
* newdf_MOD.IWV_THAAO.hvplot.scatter(x='Time2', height=height, 
                                     width=width, size=size, 
                                     color=color_THAAO)
* newdf_MYD.IWV_THAAO.hvplot.scatter(x='Time2',height=height, 
                                     width=width, size=size, 
                                     color=color_THAAO)).opts(title='IWV MXD-VESPA')).cols(1)

In [ ]:
# ok
title = f'IWV MXD-VESPA              deltatime={deltatime}      # data points={number_MXD_data_points_deltatime}'
(newdf_MOD_timeseries *newdf_MYD_timeseries 
* newdf_MOD.IWV_THAAO.hvplot.scatter(x='Time2', height=height, 
                                     width=width, size=size, 
                                     color=color_THAAO)
* newdf_MYD.IWV_THAAO.hvplot.scatter(x='Time2',height=height, 
                                     width=width, size=size, 
                                     color=color_THAAO)).opts(title=title, show_grid=True)

In [ ]:
(MOD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                   height=height, width=width, size=size, 
                                   color=color_MOD,title ='IWV MOD')
* MOD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* MYD_IR.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV  (cm)', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD')
* MYD_IR.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
  ).opts(title='MOD+MYD') 

### bisettrice

In [ ]:
slope = hv.Slope(1, 0).opts(line_color='red')

### Regressione con Holoviews

### **Regressione con sklearn**

In [ ]:
def regression_sklearn(x,y, color='orange', overlay_values=True):
    """
    Input: Pd.dataframe column
    overlay values: overlay of coefficients on the plot
       
    Return a list having:
    1- all regress element
    2- R coefficient
    3- slope value,
    4- intercept value """
    
    model = LinearRegression()
    x = x.values.reshape(-1,1)
    y = y.values.reshape(-1,1)
    reg = LinearRegression().fit(x,y)
    regress = hv.Slope(reg.coef_[0][0],reg.intercept_[0]).opts(color=color)
    
    if overlay_values == True:
        return [regress,#*hv.Text(max(x)-0.3,min(y)+0.1, 
                        #        f'y = {reg.coef_[0][0]} x + {reg.intercept_[0]}'),
                reg.score(x, y),
                reg.coef_[0][0],
                reg.intercept_[0]]
    else:
        return [regress,reg.score(x, y),
                reg.coef_[0][0],reg.intercept_[0]]

In [ ]:
def regression_eq_text(df_x, df_y, regress, x_offset=-0.3,y_offset=+0.1):
    return hv.Text(max(df_x)+x_offset, min(df_y)+y_offset, 
                                f'y = {regress[2]:.3f} x + {regress[3]:.3f} \n R = {regress[1]:.3f}')

In [ ]:
regress_MOD_sk = regression_sklearn(newdf_MOD.IWV_MODIS,newdf_MOD.IWV_THAAO)
regress_MOD_sk_text = regression_eq_text(newdf_MOD.IWV_MODIS,
                                         newdf_MOD.IWV_THAAO, regress_MOD_sk)
regress_MYD_sk = regression_sklearn(newdf_MYD.IWV_MODIS,
                                    newdf_MYD.IWV_THAAO, color='black')
regress_MYD_sk_text = regression_eq_text(newdf_MYD.IWV_MODIS,
                                         newdf_MYD.IWV_THAAO, regress_MYD_sk)
regress_TOT_sk = regression_sklearn(newdf_TOT.IWV_MODIS,newdf_TOT.IWV_THAAO, 
                                    color='cyan')
regress_TOT_sk_text = regression_eq_text(newdf_TOT.IWV_MODIS,
                                         newdf_TOT.IWV_THAAO, regress_TOT_sk)

### **MODIS vs THAAO Integrated Water Vapor (IWV) timeseries scatter plots**

### Scatter su differenza secondi

In [ ]:
ThaaoVsMOD = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                      height=500, width=600, c='vza',
                                      s='diff_int', scale=0.5,
                                      xlabel='IWV MOD (cm)', 
                                      ylabel='IWV THAAO (cm)'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


ThaaoVsMYD = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                      height=500, width=600, c='vza',
                                      s='diff_int', scale=0.5, 
                                      xlabel='IWV MYD (cm)', 
                                      ylabel='IWV THAAO (cm)'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(ThaaoVsMOD*slope*regress_MOD_sk[0]*regress_MOD_sk_text
+ ThaaoVsMYD*slope*regress_MYD_sk[0]
 *regress_MYD_sk_text
+ ThaaoVsMOD*ThaaoVsMYD*slope*regress_TOT_sk_text
* regress_MOD_sk[0]*regress_MYD_sk[0]*regress_TOT_sk[0]).cols(2)

### Scatter su differenza distanza geografica

In [ ]:
ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=0.3,
                                              xlabel='IWV MOD (cm)', 
                                              ylabel='IWV THAAO (cm)'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=0.3,
                                              xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_sk_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_sk_text
+ ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_TOT_sk_text
 *regress_MOD_sk[0]*regress_MYD_sk[0]*regress_TOT_sk[0]).opts(width=400, height=300).cols(2)

### Scatter totale per piattaforma

In [ ]:
ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MOD  (cm)',
                                              ylabel='IWV THAAO (cm)'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD (cm)', 
                                              ylabel='IWV THAAO (cm)'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_sk_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_sk_text
+ ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_TOT_sk_text
 *regress_MOD_sk[0]*regress_MYD_sk[0]*regress_TOT_sk[0]).cols(1)

In [ ]:
# ok
height=500
width=700
title = f'IWV MXD-VESPA              deltatime={deltatime}      # data points={number_MXD_data_points_deltatime}'

ThaaoVsMOD_scatter_ok = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MXD  (cm)',
                                              ylabel='IWV VESPA (cm)')

ThaaoVsMYD_scatter_ok = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale, 
                                              #xlabel='IWV MYD (cm)', 
                                              ylabel='IWV VESPA (cm)')

(ThaaoVsMOD_scatter_ok*ThaaoVsMYD_scatter_ok
*slope
*regress_TOT_sk_text
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_TOT_sk[0]).opts(title=title, show_grid=True)

### Stagionalità

In [ ]:
xlim = [0,2]
ylim = [0,2]
regress_MOD_s_text = regression_eq_text(newdf_MOD.IWV_MODIS,
                                        newdf_MOD.IWV_THAAO, regress_MOD_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_MYD_s_text = regression_eq_text(newdf_MYD.IWV_MODIS,
                                         newdf_MYD.IWV_THAAO, regress_MYD_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_TOT_s_text = regression_eq_text(newdf_TOT.IWV_MODIS,
                                         newdf_TOT.IWV_THAAO, regress_TOT_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)

ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, 
                                              c='vza', 
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=0.3, 
                                              xlabel='IWV MOD (cm)', 
                                              ylabel='IWV THAAO (cm)',
                                              groupby=['time.month'])

ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, c='vza',
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=0.3, 
                                              xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              groupby=['time.month'])

(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_s_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_s_text
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_TOT_s_text
   *regress_MOD_sk[0]*regress_MYD_sk[0]*regress_TOT_sk[0]
  ).opts(title='DDDDD', xlabel= 'IWV MXD (cm)')).cols(1)

### GroupBy month

In [ ]:
ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,xlim=[0,2],ylim=[0,2],
                                              scale=0.3, xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'], groupby=['time.month'])

ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, xlim=[0,2],ylim=[0,2],
                                              scale=0.3, xlabel='IWV MOD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'], groupby=['time.month'])

(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_s_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_s_text
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_TOT_sk[0]*regress_TOT_s_text
  ).opts(title='DDDDD', xlabel = 'IWV MXD (cm)')).cols(2)

In [ ]:
ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,xlim=[0,2],ylim=[0,2],
                                              scale=0.3, xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'], groupby=['time.month'])

ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, xlim=[0,2],ylim=[0,2],
                                              scale=0.3, xlabel='IWV MOD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'], groupby=['time.month'])

pn.panel((ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_s_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_s_text
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_TOT_sk[0]*regress_TOT_s_text
  ).opts(title='DDDDD', xlabel = 'IWV MXD (cm)')).cols(2),
         widget_type='scrubber', widget_location='top')

### By month

In [ ]:
#cmap = 'hsv'
ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, xlabel='IWV MOD (cm)',
                                              ylabel='IWV THAAO (cm)', by=['time.month'])

ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'])

(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_sk_text
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_sk_text
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_MOD_sk[0]*regress_MYD_sk[0]
   *regress_TOT_sk[0]*regress_TOT_sk_text 
  ).opts(title='DDDDD', xlabel = 'IWV MXD (cm)')).cols(1)

In [ ]:
# ok stagionalità
#cmap = 'hsv'
ThaaoVsMOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, xlabel='IWV MOD (cm)',
                                              ylabel='IWV THAAO (cm)', by=['time.month'])

ThaaoVsMYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, xlabel='IWV MYD (cm)',
                                              ylabel='IWV THAAO (cm)',
                                              by=['time.month'])


(ThaaoVsMOD_scatter*ThaaoVsMYD_scatter
# *slope
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_TOT_sk[0]*regress_TOT_sk_text
).opts(title='IWV seasonality', xlabel = 'IWV MXD (cm)', show_grid=True)

### Diff IWV MXD-THAAO vs IWV THAAO 

In [ ]:
height=380
width=1000

In [ ]:
Diff_iwv_Vs_MOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_IWV', by=['time.month'],
                                                   xlabel='IWV VESPA (cm)',
                                                   ylabel='Diff MOD-VESPA (cm)',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diff_iwv_Vs_MYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_IWV', by=['time.month'],
                                                   xlabel='IWV VESPA (cm)',
                                                   ylabel='Diff MYD-VESPA (cm)',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diff_iwv_Vs_MOD_scatter
+ Diff_iwv_Vs_MYD_scatter
+ (Diff_iwv_Vs_MOD_scatter*Diff_iwv_Vs_MYD_scatter).opts(ylabel='Diff MXD-VESPA (cm)')).cols(1)

In [ ]:
title = f'IWV MXD-VESPA              deltatime={deltatime}      # data points={number_MXD_data_points_deltatime}'
((Diff_iwv_Vs_MOD_scatter*Diff_iwv_Vs_MYD_scatter).
 opts(title=title, ylabel='Diff MXD-VESPA (cm)'))

### Diff % IWV MXD-THAAO vs IWV THAAO 

In [ ]:
Diffperc_iwv_Vs_MOD_scatter = newdf_MOD.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['time.month'],
                                                   xlabel='IWV VESPA (cm)',
                                                   ylabel='100*(MOD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diffperc_iwv_Vs_MYD_scatter = newdf_MYD.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['time.month'],
                                                   xlabel='IWV VESPA (cm)',
                                                   ylabel='100*(MYD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diffperc_iwv_Vs_MOD_scatter
+ Diffperc_iwv_Vs_MYD_scatter
+ (Diffperc_iwv_Vs_MOD_scatter*Diffperc_iwv_Vs_MYD_scatter)
 .opts(ylabel='100*(MXD-VESPA)/VESPA')).cols(1)

In [ ]:
title = f'IWV 100*(MXD-VESPA)/VESPA              deltatime={deltatime}      # data points={number_MXD_data_points_deltatime}'
((Diffperc_iwv_Vs_MOD_scatter*Diffperc_iwv_Vs_MYD_scatter)
 .opts(title=title, ylabel='100*(MXD-VESPA)/VESPA'))